In [8]:
import os
import numpy as np
import pandas as pd
import glob
from tqdm.auto import tqdm
import seaborn as sns
import random
import torch

from collections import defaultdict
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

In [9]:
from matplotlib import pyplot as plt

In [10]:
path_to_runs = '/opt/app/projects/FacadeParsing/scripts/runs'
path_to_weights = '/data/models'

In [11]:
def tabulate_events(dpath):
    summary_iterators = [EventAccumulator(os.path.join(dpath, dname)).Reload() for dname in os.listdir(dpath)]

    tags = summary_iterators[0].Tags()['scalars']

    for it in summary_iterators:
        assert it.Tags()['scalars'] == tags

    out = defaultdict(list)
    steps = []

    for tag in tags:
        steps = [e.step for e in summary_iterators[0].Scalars(tag)]

        for events in zip(*[acc.Scalars(tag) for acc in summary_iterators]):
            assert len(set(e.step for e in events)) == 1

            out[tag].append([e.value for e in events])

    return out, steps

In [12]:
def log_to_df(log_path):
    data, _ = tabulate_events(log_path)
    model_name = path.split('/')[-1]
    
    # create df
    df = pd.DataFrame({n:np.array(d).flatten() for n,d in data.items()})
    
    # put column epoch
    df = df.reset_index().rename(columns={'index': 'epoch'})
    
    # one can add columns on the dataframe such as the beta
    # put beta in column
    #beta = model_name.split('heatmaps')[-1]
    #df['beta'] = 'beta = ' + beta
    #df['beta_value'] = float(beta)
    
    return df

## From tensorboard run logs to CSV

In [13]:
for path in tqdm(sorted(glob.glob('{}/*unet*predictions-mask*heatmaps*'.format(path_to_runs)))[:-2]):
    df = log_to_df(path)
    model_name = path.split('/')[-1]
    try:
        df.to_csv('{}/{}/stats.csv'.format(path_to_weights, model_name), index=False)
    except FileNotFoundError:
        print(model_name)
    except KeyError:
        print(model_name)